In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environment import RNAInvEnvironment, make_vec_env
import torch as th

In [384]:
max_steps = 512

In [654]:
objective_structure = '(((((((....(((...........)))((((((((..(((((((((((((((((((...(((((......))))).)))))).)))))))))))))..))))))))..)))))))'
objective_structure = '(((((((((((((((.(((((.((.....)).((.....)).((.....)).((.....)).((.....)).))))).(((.....))).(((((.((.....)).((.....)).((.....)).((.....)).((.....)).))))).(((.....))).(((((.((.....)).((.....)).((.....)).((.....)).((.....)).))))).(((.....))).(((((.((.....)).((.....)).((.....)).((.....)).((.....)).))))).(((.....))).(((((.((.....)).((.....)).((.....)).((.....)).((.....)).))))).)))))))))))))))'
# n_envs=12
env = make_vec_env(RNAInvEnvironment, n_envs=n_envs, env_kwargs={'objective_structure': objective_structure, 'max_steps': max_steps, 'tuple_obs_space': True})
# env = RNAInvEnvironment(objective_structure=objective_structure, max_steps=max_steps, tuple_obs_space=True)

In [655]:
# np.array(env.observation_space.sample())

In [656]:
# np.array(env.observation_space.sample())

In [657]:
# list(env.observation_space.high)

In [658]:
env.observation_space.shape[0]

389

In [662]:
obs = env.reset()
obs.shape

(12, 389)

In [663]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.policies import ActorCriticPolicy

In [664]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import gym
import torch as th

class EmbeddinsFeatureExtractor(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 256):
        super(EmbeddinsFeatureExtractor, self).__init__(observation_space, features_dim)
        # We assume CxHxW images (channels first)
        # Re-ordering will be done by pre-preprocessing or wrapper
        EMBEDDING_DIM = 2
        self.base_embeding = th.nn.Embedding(4, EMBEDDING_DIM).cuda()
#         pair_embeding = th.nn.Embedding(6 + 1, EMBEDDING_DIM).cuda()
#         self.embeding_layers = [pair_embeding if t==6 else base_embeding for t in observation_space.high]
        self.embeding_layers = [self.base_embeding] * len(observation_space.high)

        # Compute shape by doing one forward pass
#         with th.no_grad():
#             n_flatten = self.embeding_layers(
#                 th.as_tensor(observation_space.sample()[None]).float()
#             ).shape[1]

        self.linear_1 = th.nn.Sequential(th.nn.Linear(EMBEDDING_DIM * len(self.embeding_layers), features_dim), th.nn.ReLU())
#         self.linear_2 = th.nn.Sequential(th.nn.Linear(observation_space.shape[0], features_dim), th.nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        embs_out = [self.embeding_layers[i](b) for i, b in enumerate(observations.T.int())]
        
#         return self.linear_1(th.cat(embs_out, dim=1))
        return self.linear_1(self.base_embeding(observations.int()).flatten(start_dim=1))

In [665]:
obs = env.reset()

In [670]:
efe = EmbeddinsFeatureExtractor(env.observation_space, features_dim=256).cuda()

In [671]:
obs.shape

(12, 389)

In [672]:
out = efe(th.as_tensor(obs).cuda())

In [673]:
out.shape

torch.Size([12, 256])

In [674]:
# th.as_tensor(env.observation_space.sample())

In [675]:
policy_kwargs = dict(
    features_extractor_class=EmbeddinsFeatureExtractor,
    features_extractor_kwargs=dict(features_dim=256),
)

In [676]:
model = PPO(
    ActorCriticPolicy,
    env,
    verbose=0,
    tensorboard_log='tensorboard_logs',
    n_steps=max_steps,
    gamma=0.99,
    policy_kwargs=policy_kwargs
)

In [677]:
eval_env = make_vec_env(
    RNAInvEnvironment, n_envs=1,
    env_kwargs={'objective_structure': objective_structure, 'max_steps': 2*max_steps, 'tuple_obs_space': True}
)

In [678]:
eval_callback = EvalCallback(
    eval_env = eval_env,
    eval_freq=5*max_steps,
    n_eval_episodes=5,
    deterministic=True,
    verbose=1,
    best_model_save_path='models/embiddings_diff',
)

In [679]:
%%time
model.learn(
    total_timesteps=1_000_000,
    callback=[eval_callback]
)

Eval num_timesteps=30720, episode_reward=6.34 +/- 7.41
Episode length: 1024.00 +/- 0.00
New best mean reward!
Eval num_timesteps=61440, episode_reward=17.64 +/- 20.48
Episode length: 1024.00 +/- 0.00
New best mean reward!
Eval num_timesteps=92160, episode_reward=12.00 +/- 21.09
Episode length: 1024.00 +/- 0.00
Eval num_timesteps=122880, episode_reward=23.40 +/- 14.07
Episode length: 1024.00 +/- 0.00
New best mean reward!
Eval num_timesteps=153600, episode_reward=22.38 +/- 8.11
Episode length: 1024.00 +/- 0.00
Eval num_timesteps=184320, episode_reward=25.32 +/- 13.79
Episode length: 1024.00 +/- 0.00
New best mean reward!
Eval num_timesteps=215040, episode_reward=30.86 +/- 20.48
Episode length: 1024.00 +/- 0.00
New best mean reward!
Eval num_timesteps=245760, episode_reward=33.52 +/- 14.02
Episode length: 1024.00 +/- 0.00
New best mean reward!
Eval num_timesteps=276480, episode_reward=34.12 +/- 9.57
Episode length: 1024.00 +/- 0.00
New best mean reward!
Eval num_timesteps=307200, episode

KeyboardInterrupt: 

In [34]:
import RNA
sol_1 = 'CUGCUUGGUUUGGGCCCUUUCUUUCCCCGCCCUAUACGGGGGUAAUCGUGGUCAAGGGGGUCGGUGGAUUGACCGAACUUUGAUUCACGGUUAUUUCGAUGUGGGGCAAUAAGUAG'
sol_2 = 'CGUUCGGCCAUGUCCUCGAAAAAUUGACGCUAAACGGCAUACCGGCGAUUUUUAGGCUACUCACCGAAAAGGGUGGCGCUUGACAGAUUGUUGGUGUUACGUUUGGCACCCGAAUG'
sol_3 = 'UUGACCCUGGGCGUGGUGUGGGUGAACGAGCAGUGUCUGAAUAUUUUAGUCCACCCUUGCUGGGGCCUCUAUUCUAUAGGGUGAGGUUGGAAUAUUUUAAUAUUGUUGUGGGUCGA'
sol_4 = 'GGUGGAGAAUAUCGCCUAGCCCUAACGAGCGCGCAUAUAUAGGCUUUUGACGGUGUGCAGCAGAGUGGACCCUCUGAUACGCUAGUUGGAGGCUUGUUAGUGUGUGUAACUCCACC'

objective_structure_simple_1 = '(((((((....(((...........)))((((((((..(((((((((((((((((((...(((((......))))).)))))).)))))))))))))..))))))))..)))))))'

sec_struct, energy = RNA.fold(sol_3)
sec_struct == objective_structure_simple_1

True

In [136]:
sec_struct == objective_structure

True

In [ ]:
model.learn(total_timesteps=25000)
model.save("ppo_cartpole")